# A model for math and reading proficiency in US school districts from 2009-2017

## Introduction

Individuals with low reading proficiency are more likely to drop-out of high school, to end up in the criminal justice system, and to live in poverty. Even more, the lack of high school education is a powerful predictor of mortality variation among US states and high school dropouts make up disproportionately higher percentages of the nation’s institutionalized population than of the nation’s noninstitutionalized population, so low achievement in math and reading proficiency could be the start of a much larger issue.

To understand the different levels of proficiency, we used features X, Y, Z in various models with the intent to answer the question of how does instructional spend from 2009-2016 spend impact math and reading proficiency amongst different demographics (e.g., race, gender, and socioeconomic status), as we hypothesize that achievement is determined by instructional expenditure.

Our findings show that: i) xxxxxxxx, ii) xxxxxxx, and iii) xxxxxx. 


## Education Data Models

This notebook contains two models that predict educational outcomes based on instructional spend per district from 2009 to 2018. We use using publically available data by the Education Department.
The following featues are included:
- gender
- grade
- race

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Financial Data
This dataframe  contains district level finance data including revenues from federal, state, and local governments and expenditures.

In [3]:
finance_df = pd.read_csv("financial/districts_ccd_finance.csv.gz",compression='gzip')
finance_df.head(5)

# only keep columns we are interested in
finance_df = finance_df[['year', 'leaid', 'fips', 'censusid', 'rev_total', 'exp_total', 
                                             'exp_current_instruction_total', 'enrollment_fall_responsible']]

finance_df = finance_df[finance_df.year > 2008]

finance_df.head(5)

,year,leaid,fips,censusid,rev_total,exp_total,exp_current_instruction_total,enrollment_fall_responsible
258542,2009,0100002,1.0,-2.000000e+00,-2.0,-2.0,-2.0,0.0
258543,2009,0100005,1.0,1.504840e+12,38927000.0,57286000.0,20908000.0,4104.0
258544,2009,0100006,1.0,1.504800e+12,57766000.0,57101000.0,29908000.0,5777.0
258545,2009,0100007,1.0,1.503740e+12,149924000.0,152411000.0,83430000.0,12889.0
258546,2009,0100008,1.0,1.504530e+12,85030000.0,85734000.0,44662000.0,8654.0


## District Directory Data
This dataframe contains school district geographic and mailing information, agency type, highest and lowest grades offered, special education students and English language learners, and full-time equivalent teachers and other staff.

In [4]:
directory_2009_df = pd.read_csv("district_directory/school-districts_lea_directory_2009.csv", compression='gzip')
directory_2010_df = pd.read_csv("district_directory/school-districts_lea_directory_2010.csv", compression='gzip')
directory_2011_df = pd.read_csv("district_directory/school-districts_lea_directory_2011.csv", compression='gzip')
directory_2012_df = pd.read_csv("district_directory/school-districts_lea_directory_2012.csv", compression='gzip')
directory_2014_df = pd.read_csv("district_directory/school-districts_lea_directory_2014.csv", compression='gzip')
directory_2015_df = pd.read_csv("district_directory/school-districts_lea_directory_2015.csv", compression='gzip')
directory_2016_df = pd.read_csv("district_directory/school-districts_lea_directory_2016.csv", compression='gzip')
directory_2017_df = pd.read_csv("district_directory/school-districts_lea_directory_2017.csv", compression='gzip')
directory_2018_df = pd.read_csv("district_directory/school-districts_lea_directory_2018.csv", compression='gzip')

In [5]:
# missing concatenation in overall directory df
directory_df = pd.concat([directory_2009_df, directory_2010_df, directory_2011_df, directory_2012_df,
                         directory_2013_df, directory_2014_df, directory_2015_df, directory_2016_df,
                         directory_2017_df, directory_2018_df], axis=0)
directory_df.head(5)

NameError: name 'directory_2013_df' is not defined

In [ ]:
directory_df = directory_df[['year', 'leaid', 'lea_name', 'state_leaid','state_location', 'zip_location', 'fips', 
                             'county_code', 'county_name', 'latitude', 'longitude', 'urban_centric_locale']]

In [ ]:
directory_df.head(5)

In [ ]:
edfacts_2009_df = pd.read_csv("district_edfacts/districts_edfacts_assessments_2009.csv", compression='gzip')
edfacts_2010_df = pd.read_csv("district_edfacts/districts_edfacts_assessments_2010.csv", compression='gzip')
edfacts_2011_df = pd.read_csv("district_edfacts/districts_edfacts_assessments_2011.csv", compression='gzip')
edfacts_2012_df = pd.read_csv("district_edfacts/districts_edfacts_assessments_2012.csv", compression='gzip')
edfacts_2013_df = pd.read_csv("district_edfacts/districts_edfacts_assessments_2013.csv", compression='gzip')
edfacts_2014_df = pd.read_csv("district_edfacts/districts_edfacts_assessments_2014.csv", compression='gzip')
edfacts_2015_df = pd.read_csv("district_edfacts/districts_edfacts_assessments_2015.csv", compression='gzip')
edfacts_2016_df = pd.read_csv("district_edfacts/districts_edfacts_assessments_2016.csv", compression='gzip')
edfacts_2017_df = pd.read_csv("district_edfacts/districts_edfacts_assessments_2017.csv", compression='gzip')

In [ ]:

edfacts_df = pd.concat([edfacts_2009_df, edfacts_2010_df, edfacts_2011_df, edfacts_2012_df,
                         edfacts_2013_df, edfacts_2014_df, edfacts_2015_df, edfacts_2016_df,
                         edfacts_2017_df], axis=0)
edfacts_df.head(5)

In [ ]:
print(edfacts_df.columns.tolist())

In [ ]:
edfacts_df = edfacts_df[['leaid', 'leaid_num', 'year', 'lea_name', 'fips', 'grade_edfacts', 'race', 'sex', 'lep', 
                         'homeless', 'migrant', 'disability', 'econ_disadvantaged', 'foster_care', 'military_connected', 
                         'read_test_num_valid', 'read_test_pct_prof_low', 'read_test_pct_prof_high', 
                         'read_test_pct_prof_midpt', 'math_test_num_valid', 'math_test_pct_prof_low', 
                         'math_test_pct_prof_high', 'math_test_pct_prof_midpt']]

### Not sure we need enrollment df when race is included in edfacts (I'm using it, but calling it out)

In [9]:
enrollment_2009_df = pd.read_csv("district_enrollment/schools_ccd_lea_enrollment_2009.csv", compression='gzip')
enrollment_2010_df = pd.read_csv("district_enrollment/schools_ccd_lea_enrollment_2010.csv", compression='gzip')
enrollment_2011_df = pd.read_csv("district_enrollment/schools_ccd_lea_enrollment_2011.csv", compression='gzip')
enrollment_2012_df = pd.read_csv("district_enrollment/schools_ccd_lea_enrollment_2012.csv", compression='gzip')
enrollment_2013_df = pd.read_csv("district_enrollment/schools_ccd_lea_enrollment_2013.csv", compression='gzip')
enrollment_2014_df = pd.read_csv("district_enrollment/schools_ccd_lea_enrollment_2014.csv", compression='gzip')
enrollment_2015_df = pd.read_csv("district_enrollment/schools_ccd_lea_enrollment_2015.csv", compression='gzip')
enrollment_2016_df = pd.read_csv("district_enrollment/schools_ccd_lea_enrollment_2016.csv", compression='gzip')
enrollment_2017_df = pd.read_csv("district_enrollment/schools_ccd_lea_enrollment_2017.csv", compression='gzip')
enrollment_2018_df = pd.read_csv("district_enrollment/schools_ccd_lea_enrollment_2018.csv", compression='gzip')

In [10]:

enroll_df = pd.concat([enrollment_2009_df, enrollment_2010_df, enrollment_2011_df, enrollment_2012_df,
                         enrollment_2013_df, enrollment_2014_df, enrollment_2015_df, enrollment_2016_df,
                         enrollment_2017_df, enrollment_2018_df], axis=0)
enroll_df.head(5)

,year,leaid,fips,grade,race,sex,enrollment
0,2009,100002,1,3,1,1,0.0
1,2009,100002,1,3,1,2,0.0
2,2009,100002,1,3,1,99,0.0
3,2009,100002,1,3,2,1,0.0
4,2009,100002,1,3,2,2,0.0


In [6]:
print(enroll_df.columns.tolist())

NameError: name 'enroll_df' is not defined

In [7]:
# merging directory, edfacts
master_temp_df = pd.merge(directory_df, edfacts_df)

NameError: name 'directory_df' is not defined

In [ ]:
# merging (directory, edfacts) with enroll
master_temp2_df = pd.merge(master_temp_df, enroll_df)

In [ ]:
master_df = pd.merge(master_temp2_df, finance_df)
# Todo: refactor to use join. 

In [ ]:
print(master_df.columns.tolist())

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Adding Instructional Spend Per Student Feature

In [ ]:
master_df["total_spend_per_student"] = master_df[ "exp_total"] / master_df[ "enrollment_fall_responsible"]
master_df["instruction_spend_per_student"] = master_df[ "exp_current_instruction_total"] / master_df[ "enrollment_fall_responsible"]

# Clean Data Frame
0. Remove NaN
1. Only keep positive numbers,  exclude lat, long
2. Remove outliers for  instructional total  
3. Remove districts that have less than 100 students

In [ ]:
master_df_cleaned = master_df.dropna()

def cleanup_data(dataframe):
    """Clean up dataframe returns a dataframe"""
    
    numerical_columns = ['grade_edfacts', 'race', 'sex', 'lep', 
                         'homeless', 'migrant', 'disability', 
                         'econ_disadvantaged', 'foster_care', 
                         'military_connected', 'read_test_num_valid',
                         'read_test_pct_prof_low', 'read_test_pct_prof_high',
                         'read_test_pct_prof_midpt', 'math_test_num_valid', 
                         'math_test_pct_prof_low', 'math_test_pct_prof_high', 
                         'math_test_pct_prof_midpt', 'grade', 'enrollment', "instruction_spend_per_student" , "total_spend_per_student"]
    
    
    # We only want to keep positive numbers
    for column_name in numerical_columns:
        dataframe =  dataframe[(dataframe[column_name] >= 0) ]
        
    # Remove outliers for instructional total   
    #dataframe[dataframe['exp_current_instruction_total'] < float('1.646168e+09')]    
    
    # Only include districts with more than 100 students 
    #dataframe = dataframe[dataframe['enrollment_fall_responsible'] > 50]
    
    return dataframe

cleaned_df = cleanup_data(master_df_cleaned)
cleaned_df.head()

# Box Plot
## Differences between sexes in reading proficiency

In [ ]:
sns.boxplot(x='sex', y = 'read_test_pct_prof_midpt', data = cleaned_df) 
plt.title("Gender vs Reading proficiency mid");
# 1 is male 
# 2 is female
# 99 is Total 

## Differences between Races in reading proficiency

In [ ]:
sns.boxplot(x='race', y = 'read_test_pct_prof_midpt', data = grade_12_df) 
plt.title("Race vs Read Prof mid");
"""""
1—White
2—Black
3—Hispanic
4—Asian
5—American Indian or Alaska Native
6—Native Hawaiian or other Pacific Islander
7—Two or more races
8—Nonresident alien
9—Unknown
20—Other
99—Total
"""""

In [ ]:
# T-test Male and females 

#https://www.statisticshowto.datasciencecentral.com/probability-and-statistics/t-test/
from   scipy import stats

## A simple t test for difference of means
t2, p2 = stats.ttest_ind(cleaned_df.loc[cleaned_df['sex'] == 1, 'read_test_pct_prof_midpt']
                         ,cleaned_df.loc[cleaned_df['sex'] == 2, 'read_test_pct_prof_midpt'])

print("Reading Test are the Same for Male and females")
print("t = " + str(t2))
print("p = " + str(p2))


In [ ]:
# Instructional Spend Box Plot

sns.boxplot(x='race', y = 'instruction_spend_per_student', data = cleaned_df) 
plt.title("instruction_spend_per_student vs race");


## Correlation matrix

To find the variables that have the greatest influence on read_test_pct_prof_midpt, we can compute a <b>correlation matrix</b> which measures the pairwise correlations between any two variables:

In [ ]:
corr_mat  = cleaned_df[[ "total_spend_per_student", "instruction_spend_per_student" ,'grade_edfacts', 'race', 'sex', 'lep', 'homeless', 'migrant', 'disability', 'econ_disadvantaged','read_test_num_valid', 'read_test_pct_prof_low', 'read_test_pct_prof_high', 'read_test_pct_prof_midpt', 'math_test_num_valid', 'math_test_pct_prof_low', 'math_test_pct_prof_high', 'math_test_pct_prof_midpt']].corr()
corr_mat

In [ ]:
sns.heatmap(corr_mat,cmap=sns.diverging_palette(220, 10, as_cmap=True))
plt.title("Correlation Matrix")

## Using linear models to account for variables correlated with reading proficiency

Once we identify some independent variables that are correlated with the dependent variable, a linear model can be used to capture this relationship quantitatively. A linear model does this by finding a line that [**best fits**](https://mathbits.com/MathBits/TISection/Statistics1/LineFit.htm) the data points:

In [ ]:
sns.lmplot(x = 'instruction_spend_per_student', y = 'read_test_pct_prof_midpt', data = cleaned_df, scatter_kws = {'color': (174/255,199/255,14/255)})
plt.title("instruction_spend_per_student vs. read_test_pct_prof_midpt", fontsize=20, verticalalignment='bottom')
plt.xlabel("instruction_spend_per_student")
plt.ylabel("read_test_pct_prof_midpt")

## OLS Model for Reading proficiency only Grade 12

In [ ]:
grade_12_df = cleaned_df[cleaned_df["grade"] == 12]
x = grade_12_df[["instruction_spend_per_student", 'lep', 'homeless', 'migrant', 'disability', 'econ_disadvantaged', 'foster_care', 'military_connected','race',"sex",'exp_current_instruction_total']]  # add more explanatory variables; 
y = grade_12_df['read_test_pct_prof_midpt']
x = sm.add_constant(x)
est = sm.OLS(y, x).fit()
est.summary()

## OLS Model for Reading proficiency only (math excluded)

In [ ]:
x = cleaned_df[['lep', 'homeless', 'migrant', 'disability', 
                'econ_disadvantaged', 'foster_care', 
                'military_connected','race',"sex",
                'exp_current_instruction_total']]  

y = cleaned_df['read_test_pct_prof_midpt']
x = sm.add_constant(x)
est = sm.OLS(y, x).fit()
est.summary()

In [ ]:
# getting one single year to better understand how the model is working
finance_2015_df = finance_df[finance_df.year == 2015]

In [ ]:
# getting one single year to better understand how the model is working
finance_2015_df.head(5)

In [ ]:
# getting one single year to better understand how the model is working
edfacts_2015_df.head(5)

In [ ]:
# this would be disagregated by race (ie, race is not 00)
edfacts_2015_by_race_df = edfacts_2015_df[(edfacts_2015_df['sex']== 99)
                                       & (edfacts_2015_df['lep']== 99)
                                      & (edfacts_2015_df['homeless']== 99)
                                      & (edfacts_2015_df['migrant']== 99)
                                      & (edfacts_2015_df['disability']== 99)
                                      & (edfacts_2015_df['econ_disadvantaged']== 99)
                                      & (edfacts_2015_df['foster_care']== 99)
                                      & (edfacts_2015_df['military_connected']== 99) 
                                       & (edfacts_2015_df['grade_edfacts']== 99)]

edfacts_2015_without_total_race_df = edfacts_2015_by_race_df[edfacts_2015_by_race_df['race'] != 99]
edfacts_2015_without_total_race_df.head(20)

### For the output above, we see that if the number of valid tests taken is larger than 300, we get the actual percentage of students that scored proficient. If it is smaller than 300, it gets bucketed into ranges for anonimization.

In [ ]:
from collections import defaultdict

In [ ]:
edfacts_by_race_df = [edfacts_2009_df, edfacts_2010_df, edfacts_2011_df, edfacts_2012_df,
                         edfacts_2013_df, edfacts_2014_df, edfacts_2015_df, edfacts_2016_df,
                         edfacts_2017_df]

edfacts_df_dict = defaultdict(list)
i = 2009
for df in edfacts_by_race_df:
    new_df_name = df[(df['sex']== 99)
                                       & (df['lep']== 99)
                                      & (df['homeless']== 99)
                                      & (df['migrant']== 99)
                                      & (df['disability']== 99)
                                      & (df['econ_disadvantaged']== 99)
                                      & (df['foster_care']== 99)
                                      & (df['military_connected']== 99) 
                                       & (df['grade_edfacts']== 99)]
    edfacts_df_dict[i] = new_df_name
    i += 1


In [ ]:
edfacts_df_dict.keys()

In [ ]:
edfacts_2015_race_df = edfacts_df_dict[2015]
edfacts_2015_race_df.head()

In [ ]:
edfacts_2015_race_df['read_test_num_valid'].describe()

In [ ]:
edfacts_2015_race_df['read_test_num_valid'].describe()
edfacts_2015_race_df.hist(column='read_test_num_valid', bins=5)
edfacts_2015_race_df['read_test_num_valid']

In [ ]:
edfacts_2015_race_df_drop_read_outlier = edfacts_2015_race_df[(edfacts_2015_race_df['read_test_num_valid'] < float('1000')) 
                                                              & (edfacts_2015_race_df['race'] == 99)]
edfacts_2015_race_df_drop_read_outlier['read_test_num_valid'].describe()
edfacts_2015_race_df_drop_read_outlier.hist(column='read_test_num_valid', bins=100)

In [ ]:
# removing all negative entries, which means data is supressed (-1), not applicable (-2), or missing (-3)
# we are only focusing on districts w/ over 100 students

finance_by_year_df = [edfacts_2009_df, edfacts_2010_df, edfacts_2011_df, edfacts_2012_df,
                         edfacts_2013_df, edfacts_2014_df, edfacts_2015_df, edfacts_2016_df,
                         edfacts_2017_df]

my_values = defaultdict(list)
i = 2009
for df in edfacts_by_race_df:
    new_df_name = df[(df['sex']== 99)
                                       & (df['lep']== 99)
                                      & (df['homeless']== 99)
                                      & (df['migrant']== 99)
                                      & (df['disability']== 99)
                                      & (df['econ_disadvantaged']== 99)
                                      & (df['foster_care']== 99)
                                      & (df['military_connected']== 99) 
                                       & (df['grade_edfacts']== 99)]
    my_values[i] = new_df_name
    i += 1



finance_2015_clean_df1 = finance_2015_df[(finance_2015_df['rev_total'] > 0) 
                                       & (finance_2015_df['exp_total'] > 0)
                                       & (finance_2015_df['exp_current_instruction_total']> 0)
                                      & (finance_2015_df['enrollment_fall_responsible'] > 100)]

In [ ]:
finance_2015_clean_df1.head(5)

In [ ]:
finance_2015_clean_df1.sort_values(by = 'enrollment_fall_responsible', ascending=True)

In [ ]:
# removing all negative entries, which means data is supressed (-1), not applicable (-2), or missing (-3)
edfacts_2015_clean_df1 = edfacts_2015_race_df[(edfacts_2015_race_df['read_test_pct_prof_low'] > 0) 
                                       | (edfacts_2015_race_df['read_test_pct_prof_high'] > 0)
                                       | (edfacts_2015_race_df['read_test_pct_prof_midpt']> 0)
                                      | (edfacts_2015_race_df['math_test_pct_prof_low'] > 0)
                                      | (edfacts_2015_race_df['math_test_pct_prof_high'] > 0)
                                      | (edfacts_2015_race_df['math_test_pct_prof_midpt'] > 0)]

In [ ]:
edfacts_2015_clean_df1.head(5)

In [ ]:
df_2015_clean = pd.merge(finance_2015_clean_df1, edfacts_2015_clean_df1, on = "leaid", how = "inner")

In [ ]:
df_2015_clean.head(5)

In [ ]:
print(df_2015_clean.columns.tolist())

In [ ]:
df_2015_clean_dropna = df_2015_clean.dropna()

In [ ]:
#print(df_2015_clean_dropna)

In [ ]:
# X = inputs for prediction; Y = what we are predicting
x = df_2015_clean_dropna['exp_current_instruction_total']
y = df_2015_clean_dropna['read_test_pct_prof_high']
x = sm.add_constant(x)
est = sm.OLS(y, x).fit()
est.summary()

In [ ]:
plt.scatter(df_2015_clean_dropna['exp_current_instruction_total'], df_2015_clean_dropna['read_test_pct_prof_high'])

In [ ]:
#check the outlier and drop it
df_2015_clean_dropna.sort_values(by='exp_current_instruction_total', ascending=False)


In [ ]:
df_2015_clean_drop_outlier = df_2015_clean_dropna[df_2015_clean_dropna['exp_current_instruction_total'] < float('1.646168e+09')]

In [ ]:
plt.scatter(df_2015_clean_drop_outlier['exp_current_instruction_total'], df_2015_clean_drop_outlier['read_test_pct_prof_high'])

In [ ]:
# create new column with per capita instructional expenditure per student
df_2015_clean_drop_outlier['instructional_exp_per_student'] = df_2015_clean_drop_outlier['exp_current_instruction_total'] / df_2015_clean_drop_outlier['enrollment_fall_responsible']

In [ ]:
plt.scatter(df_2015_clean_drop_outlier['instructional_exp_per_student'], df_2015_clean_drop_outlier['read_test_pct_prof_high'])

In [ ]:
# what we see here -1.0 is not really a negative number it is $100M

In [ ]:
df_2015_clean_drop_outlier['enrollment_fall_responsible'].describe()

In [ ]:

df_2015_clean_drop_outlier.sort_values(by = 'enrollment_fall_responsible', ascending=True)

In [ ]:
# X = inputs for prediction; Y = what we are predicting
x = df_2015_clean_drop_outlier['exp_current_instruction_total']
y = df_2015_clean_drop_outlier['read_test_pct_prof_high']
x = sm.add_constant(x)
est = sm.OLS(y, x).fit()
est.summary()

In [ ]:
x = df_2015_clean_drop_outlier['exp_current_instruction_total'].tolist() 
y = df_2015_clean_drop_outlier['read_test_pct_prof_high'].tolist() 
plt.scatter(x, y) 
  
# finding the maximum and minimum 
# values of x, to get the 
# range of data 
max_x = df_2015_clean_drop_outlier['exp_current_instruction_total'].max() 
min_x = df_2015_clean_drop_outlier['read_test_pct_prof_high'].min() 
  
# range of values for plotting 
# the regression line 
x = np.arange(min_x, max_x, 1) 
  
# the substituted equation 
y = -3.186e-08 * x - 55.9731
  
# plotting the regression line 
plt.plot(y, 'r') 
plt.show() 

## Open questions:
- I'm failing to add race. Not sure what needs to be done before I can use it since it is a categorical variable
- Need to decide on the features we want (correlation amongst them?)

## Features:
- From Finance:
    - All the dollar amounts here are highly correlated; we likely don't want to get rid of them but they will be highly correlated. We can devide exp_current_instruction total by exp_total to decouple it from the total amount.
    - 'rev_total' - Numerical/measurement
    - 'exp_total'  - Numerical/measurement 
    - 'exp_current_instruction_total' - Numerical/measurement 
    - 'enrollment_fall_responsible' - Numerical/measurement   - get per capita measure on $ spend
    
- From Directory: For this we should be able to recode
    - 'urban_centric_locale': - categorical
        1—Large city = city
        2—Midsize city = city
        3—Urban fringe of large city = city
        4—Urban fringe of midsize city = city
        5—Large town = town
        6—Small town = town
        7—Rural, outside CBSA = rural
        8—Rural, inside CBSA  = rural
        9—Not assigned
        11—City, large = city
        12—City, midsize = city
        13—City, small = city
        21—Suburb, large = suburb
        22—Suburb, midsize = suburb
        23—Suburb, small = suburb
        31—Town, fringe = town
        32—Town, distant = town
        33—Town, remote = town
        41—Rural, fringe = rural
        42—Rural, distant = rural
        43—Rural, remote = rural
        
- From edfacts: 
        'grade_edfacts' -  --> Ask OhKyu about this one. For this one, the recommendation is to limit it to school districts with the same grades, otherwise, it will introduce bias. We have to try if the better prediction is by grade (potentially more noisy, and lower grades may be less predictive) or by total. REgardless of which one we chose, we need to be consistent
        'race', 'sex', 'lep', 
         'homeless', 'migrant', 'disability', 'econ_disadvantaged', 'foster_care', 'military_connected'
         
- From enrollment:
        - grade
        - race
        - sex 
        - enrollment
        
- Look into Kappa statistic (provides the percent of accordance and it substracts a number that it happened by chance) and chi-square test of independence (this would allow to compare more than two features). Even for categorical variables, a correlation would still get us a number

         

        



In [ ]:
# eta-squared, or eta, as an effect-size measure for the relationship of a categorical variable and a continuous variable.
# -> categorical (multi) + continuous 

def correlation_ratio(categories, measurements):
        fcat, _ = pd.factorize(categories)
        cat_num = np.max(fcat)+1
        y_avg_array = np.zeros(cat_num)
        n_array = np.zeros(cat_num)
        for i in range(0,cat_num):
            cat_measures = measurements[np.argwhere(fcat == i).flatten()]
            n_array[i] = len(cat_measures)
            y_avg_array[i] = np.average(cat_measures)
        y_total_avg = np.sum(np.multiply(y_avg_array,n_array))/np.sum(n_array)
        numerator = np.sum(np.multiply(n_array,np.power(np.subtract(y_avg_array,y_total_avg),2)))
        denominator = np.sum(np.power(np.subtract(measurements,y_total_avg),2))
        if numerator == 0:
            eta = 0.0
        else:
            eta = numerator/denominator
        return eta

In [ ]:
# This is for Pearson correlation, which may not be applicable for our type of variables. 

correlations = smokers[
    (smokers["charges"] >= charges.quantile(q=0.25))
    & (smokers["charges"] <= charges.quantile(q=0.75))
]
corr = central.corr()
corr.style.background_gradient(cmap="coolwarm").set_precision(2)